In [31]:
import requests
import json
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
AUTHENTICATION = config['WeaG']['AUTHENTICATION']



class WeaG():
    url1 = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0003-001'
    url2 = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0001-001'
    url3 = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0002-001'

    def grab(self , StationName):

        params = {'Authorization':AUTHENTICATION,
                    'StationName':StationName}
        info = {}
        #rain data
        r = requests.get(__class__.url3 , params=params)#可以寫 WeaG.url3 or __class__.url3 or self.url3 但__class__最好
        if r.json()['records']['Station']:
            info['R'] = r.json()['records']['Station'][0]['RainfallElement']['Now']['Precipitation']
        else:
            print('沒有雨量資訊')
        #------------------------------------------------------------------------------------
        r = requests.get(__class__.url1, params=params)
        if not r.json()['records']['Station']:     
            print('url1沒有數據,爬取url2')
            r = requests.get(__class__.url2, params=params) 
 
        if r.json()['records']['Station']:
            info['O'] = r.json()['records']['Station'][0]['ObsTime']['DateTime']
            info['T'] = float((r.json()['records']['Station'][0]['WeatherElement']['AirTemperature']))
            info['H'] = float(r.json()['records']['Station'][0]['WeatherElement']['RelativeHumidity'])/100
        return info

w = WeaG()

w.grab('臺北')

{'R': 0.0, 'O': '2024-08-28T14:40:00+08:00', 'T': 32.0, 'H': 0.71}

In [24]:
from threading import Thread
import requests
import json




class WeaG:

    
    def __init__(self):
        self.env = json.load(open('env.json'))
        self.url = self.env['url']
        self.AUTHENTICATION = self.env['token']

        
    
    def grab(self , site):
        '''this is a main funtion for grabbing data from CWA's open data api ->
        {O:....}'''


        def _grab(url, site ):
            nonlocal info 

            params = {'Authorization':self.AUTHENTICATION,
                            'StationName':site}
            
            r = requests.get(url, params=params)
            if r.json()['records']['Station']:
                s = r.json()['records']['Station'][0]
                info['O'] = r.json()['records']['Station'][0]['ObsTime']['DateTime']
                if 'WeatherElement' in s :

                    info['T'] = float((r.json()['records']['Station'][0]['WeatherElement']['AirTemperature']))
                    info['H'] = float(r.json()['records']['Station'][0]['WeatherElement']['RelativeHumidity'])/100
                elif 'RainfallElement' in s :
                    info['R'] = r.json()['records']['Station'][0]['RainfallElement']['Now']['Precipitation']
        

        info = {}
        ths = [None] * len(self.url)
        for i in range(len(self.url)):
            ths[i] = Thread(target=_grab, args=(self.url[i],site),daemon=True)
            ths[i].start()
            ths[i].join()
        return info

In [25]:
w.url

['https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0003-001',
 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0001-001',
 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0002-001']

In [26]:
w= WeaG()
w.grab('臺北')

{'O': '2024-08-28T16:50:00+08:00', 'T': 31.2, 'H': 0.74, 'R': 0.0}

In [29]:
from threading import Thread
from random import randint
from time import sleep , time

def test(i,j):
    global r
    a = randint(0,5)
    sleep(a)
    r[i] =  a

ths = [None] * 10   
r = [None] * 10 



for i in range(10):
    ths[i] = Thread(target=test,args=(i,i+1), daemon=True)
    ths[i].start()

start = time()
timeout = 2
for i in range(10):
    t = time() - start
    if t > timeout:
        break
    ths[i].join()

for i in range(10):
    print(r[i])


1
4
1
1
2
0
3
2
1
3


In [27]:
import we
w = we.WeaG()
help(we.WeaG)

Help on class WeaG in module we:

class WeaG(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  grab(self, site)
 |      this is a main funtion for grabbing data from CWA's open data api ->
 |      {O:....}
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  url = ['https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0003-001'...



In [28]:
w.grab('中和')

{'O': '2024-08-28T16:50:00+08:00', 'T': 30.3, 'H': 0.83, 'R': 7.5}